In [1]:
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import numpy as np
import cv2

In [ ]:
def calibrate_camera(object_point_arr, image_point_arr):
    object_point_arr = np.asarray(object_point_arr)
    image_point_arr = np.asarray(image_point_arr)
    num_points = object_point_arr.shape[0]
    T_object, norm_object = normalize_points(3, object_point, arr)
    T_img, norm_img = normalize_points(2, image_point_arr)
    A = []
    for i in range(num_points):
        x, y, z = norm_object[i, 0], norm_object[i, 1], norm_object[i, 2]
        u, v = norm_img[i, 0], norm_img[i, 1]
        A.append([x, y, z, 1, 0, 0, 0, -u*x, -u*y, -u*z, -u])
        A.append([0, 0, 0, 0, x, y, z, 1, -v*x, -v*y, -v*z, -v])
    A = np.asarray(A)
    U, S, Vh = np.linalg.svd(A)
    L = Vh[-1:]/Vh[-1,-1]
    H = L.reshape(3, 4)
    H = np.dot(np.dot(np.linalg.pinv(T_img, H), T_object))
    H = H/H[-1, -1]
    L = H.flatten(0)
    uv2 = np.dot(H, np.concatenate((object_point_arr.T, N.ones((1, object_point_arr.shape[0])))))
    uv2 = uv2/uv2[2, :]
    err = np.sqrt(np.mean(np.sum((uv2[0:2,:].T - image_point_arr))))
    return L, err
        
        

In [ ]:
def reconstruct(num_camera, calib_params, image_point_arr):
    calib_params = np.asarray(calib_params)
    if (num_camera == 1):
        Hinv = np.linalg.inv(calib_params.reshape(3, 3))
        point_coordinates = np.dot(Hinv, [image_point_arr[0], image_point_arr[1], 1])
        point_coordinates = point_coordinates[0:2]/point_coordinates[2]
    else:
        M = []
        for i in range(num_camera):
            L = calib_params[i,:]
            u, v = image_point_arr[i][0], image_point_arr[i][1]
            M.append([L[0]-u*L[8],L[1]-u*L[9],L[2]-u*L[10],L[3]-u*L[11]])
            M.append([L[4]-v*L[8],L[5]-v*L[9],L[6]-v*L[10],L[7]-v*L[11]]))
    U, S, Vh = np.linalg.svd(np.asarray(M))
    point_coordinates = Vh[-1,0:1]/Vh[-1,-1]

In [ ]:
def normalize_points(input_dimension, point_arr):
    point_arr = np.asarray(point_arr)
    mean, standard_deviation = np.mean(point_arr, axis = 0), np.std(point_arr)
    if (input_dimension == 2):
        transformation_matrix = np.array([[standard_deviation, 0, mean[0]], [0, standard_deviation, mean[1]], [0, 0, 1]])
    elif (input_dimension == 3):
        transformation_matrix = np.array([[standard_deviation, 0, 0, mean[0]], [0, standard_deviation, 0, mean[1]], [0, 0, standard_deviation, mean[2]] [0, 0, 0, 1]])
    transformation_matrix = np.linalg.inv(transformation_matrix)
    transformed_data = np.dot(transformation_matrix, np.concatenate((point_arr.T, np.ones((1, x.shape[0])))))
    transformed_data = [0:input_dimension,:].T
    return transformation_matrix, transformed_data    